In [173]:
import pandas as pd
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [174]:
def get_mols(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
#             print(".",end="")
        except Exception as e:
            print(e,'\t',smiles,'\n')
            mol = None
        return mol

In [175]:
nms=[x[0] for x in Descriptors._descList]
print(len(nms))
mol = Chem.MolFromSmiles('CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21') #example
calc = MoleculeDescriptors.MolecularDescriptorCalculator(nms)
type(calc.CalcDescriptors(mol))

208


tuple

In [176]:
def get_descriptors(m):
        try:
            calc = MoleculeDescriptors.MolecularDescriptorCalculator(nms)
            descrs = calc.CalcDescriptors(m)
#             print(".",end="")
        except Exception as e:
            print(e,'\t',m,'\n')
            descrs = None
        return descrs

In [177]:
def process(df):
    begin = time.time()
    smiles_list = pd.Series(list(df['Canonical_SMILES']))
    descrs_list = list()
    df['Mol_Objects'] = smiles_list.map(get_mols)
    mol_list = pd.Series(list(df['Mol_Objects']))
    for mol in mol_list:
        descrs_list.append(get_descriptors(mol))
    df_temp = pd.DataFrame([x for x in descrs_list])
    df = pd.concat([df,df_temp],axis=1)
    end = time.time()
    elapsed = end - begin
    print(elapsed)
    return df

# DILI

In [178]:
df = pd.read_csv('./DILI/final/DILI_Final.csv')
df

,ID,Canonical_SMILES,DILI
0,ref1_00001,CNCC[C@@H](Oc1ccccc1C)c1ccccc1,1
1,ref1_00002,CN1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,0
2,ref1_00003,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,1
3,ref1_00004,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,1
4,ref1_00005,NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...,0
...,...,...,...
2256,ref6_01298,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,0
2257,ref6_01300,CNC[C@H](O)c1ccc(O)c(O)c1,0
2258,ref6_01301,CCCCCCCC/C=C/CCCCCCCC(=O)O,0
2259,ref6_01304,NC(N)=NCCN1CCCCCCC1,0


In [179]:
df = process(df)
df_a=df[['ID','Canonical_SMILES','DILI']]
df_b=df.loc[:,0:207]
recolumn = []
for i in range(208):
    recolumn.append(f'{nms[i]}')
df_b.columns = recolumn
df = pd.concat([df_a,df_b],axis=1)
df

59.496087312698364


,ID,Canonical_SMILES,DILI,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,CNCC[C@@H](Oc1ccccc1C)c1ccccc1,1,6.193380,0.092361,6.193380,0.092361,0.848995,255.361,234.193,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,CN1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,0,6.482112,-0.455333,6.482112,0.455333,0.722715,343.898,317.690,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,1,6.271931,0.815501,6.271931,0.815501,0.817874,314.860,291.676,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,1,6.176540,0.706466,6.176540,0.706466,0.800122,326.831,307.679,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...,0,13.195604,-0.817283,13.195604,0.137241,0.622271,426.560,396.320,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,ref6_01298,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,0,14.188117,-2.026666,14.188117,0.171241,0.433136,583.689,546.393,...,0,0,0,0,0,0,0,0,0,0
2257,ref6_01300,CNC[C@H](O)c1ccc(O)c(O)c1,0,9.480602,-0.669907,9.480602,0.180178,0.510157,183.207,170.103,...,0,0,0,0,0,0,0,0,0,0
2258,ref6_01301,CCCCCCCC/C=C/CCCCCCCC(=O)O,0,10.324075,-0.664234,10.324075,0.331507,0.290517,282.468,248.196,...,0,0,0,0,0,0,0,0,10,0
2259,ref6_01304,NC(N)=NCCN1CCCCCCC1,0,5.273144,0.205596,5.273144,0.205596,0.513143,198.314,176.138,...,0,0,0,0,0,0,0,0,0,0


In [180]:
df.to_csv('./DILI/final/DILI_DESCRS.csv')

In [181]:
mfps = pd.read_csv('./DILI/final/DILI_MFPS.csv')
descrs = pd.read_csv('./DILI/final/DILI_DESCRS.csv')
df_final = pd.merge(mfps,descrs)
df_final = df_final.loc[:,"ID":'fr_urea']
df_final

,ID,Canonical_SMILES,DILI,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,CNCC[C@@H](Oc1ccccc1C)c1ccccc1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,CN1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,ref6_01298,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2257,ref6_01300,CNC[C@H](O)c1ccc(O)c(O)c1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2258,ref6_01301,CCCCCCCC/C=C/CCCCCCCC(=O)O,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0
2259,ref6_01304,NC(N)=NCCN1CCCCCCC1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [182]:
df_final.to_csv('./DILI/final/DILI_Features.csv')

## Model-Free

In [309]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
df = pd.read_csv('./DILI/final/DILI_Features_cleaned.csv', index_col=0)
df_split = df.iloc[:,1027:]
df_split

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.193380,0.092361,6.193380,0.092361,0.848995,255.361,234.193,255.162314,100,0,...,0,0,0,0,0,0,0,0,0,0
1,6.482112,-0.455333,6.482112,0.455333,0.722715,343.898,317.690,343.170292,128,0,...,0,0,0,0,0,0,0,0,0,0
2,6.271931,0.815501,6.271931,0.815501,0.817874,314.860,291.676,314.154976,116,0,...,0,0,0,0,0,0,0,0,0,0
3,6.176540,0.706466,6.176540,0.706466,0.800122,326.831,307.679,326.129824,118,0,...,0,0,0,0,0,0,0,0,0,0
4,13.195604,-0.817283,13.195604,0.137241,0.622271,426.560,396.320,426.230728,164,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,14.188117,-2.026666,14.188117,0.171241,0.433136,583.689,546.393,583.279469,224,0,...,0,0,0,0,0,0,0,0,0,0
2257,9.480602,-0.669907,9.480602,0.180178,0.510157,183.207,170.103,183.089543,72,0,...,0,0,0,0,0,0,0,0,0,0
2258,10.324075,-0.664234,10.324075,0.331507,0.290517,282.468,248.196,282.255880,118,0,...,0,0,0,0,0,0,0,0,10,0
2259,5.273144,0.205596,5.273144,0.205596,0.513143,198.314,176.138,198.184447,82,0,...,0,0,0,0,0,0,0,0,0,0


In [310]:
scaler = MinMaxScaler()
scaler.fit(df_split)
scaled =  scaler.transform(df_split) 

In [311]:
df_scaled = pd.DataFrame(scaled)
df_scaled

,0,1,2,3,4,5,6,7,8,9,...,198,199,200,201,202,203,204,205,206,207
0,0.355380,0.839519,0.355380,0.014041,0.907059,0.033733,0.033098,0.033729,0.034226,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.371948,0.786495,0.371948,0.069219,0.770975,0.046317,0.045763,0.046246,0.044643,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.359887,0.909528,0.359887,0.123972,0.873523,0.042190,0.041817,0.042120,0.040179,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.354414,0.898972,0.354414,0.107396,0.854392,0.043891,0.044245,0.043823,0.040923,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.757172,0.751453,0.757172,0.020863,0.662734,0.058065,0.057691,0.058060,0.058036,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,0.814123,0.634369,0.814123,0.026032,0.458915,0.080398,0.080455,0.080396,0.080357,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2222,0.544003,0.765721,0.544003,0.027390,0.541915,0.023478,0.023376,0.023479,0.023810,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2223,0.592402,0.766270,0.592402,0.050395,0.305223,0.037586,0.035222,0.037583,0.040923,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384615,0.0
2224,0.302576,0.850481,0.302576,0.031255,0.545133,0.025625,0.024291,0.025626,0.027530,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [316]:
describe = df_scaled.describe().T
greater_005 = describe['std'] > 0.06
describe = describe[greater_005]
describe

,count,mean,std,min,25%,50%,75%,max
0,2226.0,0.641750,0.163958,0.0,0.592254,0.685941,0.739124,1.0
1,2226.0,0.720684,0.133735,0.0,0.680727,0.744346,0.804935,1.0
2,2226.0,0.641750,0.163958,0.0,0.592254,0.685941,0.739124,1.0
4,2226.0,0.559851,0.250249,0.0,0.380334,0.587343,0.766220,1.0
10,2226.0,0.680012,0.109137,0.0,0.600400,0.702897,0.753582,1.0
...,...,...,...,...,...,...,...,...
201,2226.0,0.013477,0.115332,0.0,0.000000,0.000000,0.000000,1.0
202,2226.0,0.014376,0.119060,0.0,0.000000,0.000000,0.000000,1.0
203,2226.0,0.015274,0.093570,0.0,0.000000,0.000000,0.000000,1.0
205,2226.0,0.012354,0.081861,0.0,0.000000,0.000000,0.000000,1.0


In [317]:
describe.to_csv('./DILI/temp/Describe.csv')

In [318]:
filter = describe['std'] > 0.06
filter.index

Int64Index([  0,   1,   2,   4,  10,  11,  12,  13,  14,  15,
            ...
            196, 197, 198, 199, 200, 201, 202, 203, 205, 207],
           dtype='int64', length=138)

In [319]:
df_filtered = df_split.iloc[:,filter.index]
df_filtered

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,qed,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,...,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_urea
0,6.193380,0.092361,6.193380,0.848995,0.124936,-0.485416,0.485416,0.124936,1.000000,1.684211,...,0,0,0,0,0,0,0,0,0,0
1,6.482112,-0.455333,6.482112,0.722715,0.115262,-0.365775,0.365775,0.115262,1.083333,1.750000,...,0,0,0,0,0,0,0,0,0,0
2,6.271931,0.815501,6.271931,0.817874,0.045791,-0.340948,0.340948,0.045791,1.000000,1.727273,...,0,0,0,0,0,0,0,0,0,0
3,6.176540,0.706466,6.176540,0.800122,0.138322,-0.353471,0.353471,0.138322,1.000000,1.782609,...,0,0,0,0,0,0,0,0,0,0
4,13.195604,-0.817283,13.195604,0.622271,0.232388,-0.492959,0.492959,0.232388,1.000000,1.687500,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,14.188117,-2.026666,14.188117,0.433136,0.280215,-0.360880,0.360880,0.280215,1.162791,1.976744,...,0,0,0,0,0,0,0,0,0,0
2257,9.480602,-0.669907,9.480602,0.510157,0.157261,-0.504260,0.504260,0.157261,1.307692,2.000000,...,0,0,0,0,0,0,0,0,0,0
2258,10.324075,-0.664234,10.324075,0.290517,0.302840,-0.481230,0.481230,0.302840,0.750000,1.200000,...,0,0,0,0,0,0,0,0,0,0
2259,5.273144,0.205596,5.273144,0.513143,0.185451,-0.370152,0.370152,0.185451,1.000000,1.571429,...,0,0,0,0,0,0,0,0,0,0


In [320]:
df_label = df.iloc[:, 0:1027]
# df_label
df_free = pd.concat([df_label, df_filtered], axis=1)
df_free

,ID,Canonical_SMILES,DILI,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_urea
0,ref1_00001,CNCC[C@@H](Oc1ccccc1C)c1ccccc1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,CN1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,ref6_01298,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2257,ref6_01300,CNC[C@H](O)c1ccc(O)c(O)c1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2258,ref6_01301,CCCCCCCC/C=C/CCCCCCCC(=O)O,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2259,ref6_01304,NC(N)=NCCN1CCCCCCC1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [321]:
df_free.to_csv('./DILI/final/DILI_Model_Free.csv')

# Acute Oral Toxicity

In [192]:
df = pd.read_csv('./Acute Oral Toxicity/final/AOT_LD50_Final.csv')
df = process(df)
df_a=df[['ID','Canonical_SMILES','LD50_minLog']]
df_b=df.loc[:,0:207]
recolumn = []
for i in range(208):
    recolumn.append(f'{nms[i]}')
df_b.columns = recolumn
df = pd.concat([df_a,df_b],axis=1)
df

80.17189908027649


,ID,Canonical_SMILES,LD50_minLog,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,-0.337242,10.074698,-0.461975,10.074698,0.059630,0.463602,157.556,153.524,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,-0.124939,10.076965,-0.458889,10.076965,0.064074,0.359544,138.126,132.078,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,-0.632201,10.036149,-0.514444,10.036149,0.015926,0.470728,139.110,134.070,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,0.257439,10.173125,-0.432528,10.173125,0.096173,0.389482,171.583,165.535,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00006,COc1ccc([N+](=O)[O-])cc1,0.361728,10.165294,-0.445000,10.165294,0.074815,0.478623,153.137,146.081,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8314,ref1_11885,c1ccccc1,0.521530,2.000000,2.000000,2.000000,2.000000,0.442628,78.114,72.066,...,0,0,0,0,0,0,0,0,0,0
8315,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,-1.522879,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,...,0,0,0,0,0,0,0,0,0,0
8316,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,-1.036212,5.684815,-2.768105,5.684815,0.221566,0.717616,304.352,283.184,...,0,0,0,0,0,0,0,0,0,0
8317,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,-0.008774,12.278519,-3.690648,12.278519,0.181768,0.473671,327.572,309.428,...,0,0,0,0,0,0,0,0,0,0


In [193]:
df.to_csv('./Acute Oral Toxicity/temp2/AOT_LD50_DESCRS.csv')

In [212]:
mfps = pd.read_csv('./Acute Oral Toxicity/final/AOT_LD50_MFPS.csv')
descrs = pd.read_csv('./Acute Oral Toxicity/temp2/AOT_LD50_DESCRS.csv')
df_final = pd.merge(mfps,descrs)
df_final = df_final.iloc[:,1:]
df_final = df_final.dropna()
df_final

,ID,Canonical_SMILES,LD50_minLog,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,-0.337242,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,-0.124939,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,-0.632201,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,0.257439,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00006,COc1ccc([N+](=O)[O-])cc1,0.361728,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8314,ref1_11885,c1ccccc1,0.521530,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8315,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,-1.522879,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8316,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,-1.036212,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8317,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,-0.008774,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
df_final.to_csv('./Acute Oral Toxicity/final/AOT_LD50_Features.csv')

In [214]:
df = pd.read_csv('./Acute Oral Toxicity/final/AOT_LD50_Features.csv', index_col=0)
df_split = df.iloc[:,1027:]
df_split

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,10.074698,-0.461975,10.074698,0.059630,0.463602,157.556,153.524,156.993056,52,0,...,0,0,0,0,0,0,0,0,0,0
1,10.076965,-0.458889,10.076965,0.064074,0.359544,138.126,132.078,138.042927,52,0,...,0,0,0,0,0,0,0,0,0,0
2,10.036149,-0.514444,10.036149,0.015926,0.470728,139.110,134.070,139.026943,52,0,...,0,0,0,0,0,0,0,0,0,0
3,10.173125,-0.432528,10.173125,0.096173,0.389482,171.583,165.535,171.008706,58,0,...,0,0,0,0,0,0,0,0,0,0
4,10.165294,-0.445000,10.165294,0.074815,0.478623,153.137,146.081,153.042593,58,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8314,2.000000,2.000000,2.000000,2.000000,0.442628,78.114,72.066,78.046950,30,0,...,0,0,0,0,0,0,0,0,0,0
8315,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,219.945901,66,0,...,0,0,0,0,0,0,0,0,0,0
8316,5.684815,-2.768105,5.684815,0.221566,0.717616,304.352,283.184,304.101050,108,0,...,0,0,0,0,0,0,0,0,0,0
8317,12.278519,-3.690648,12.278519,0.181768,0.473671,327.572,309.428,326.000829,104,0,...,0,0,0,0,0,0,0,0,0,0


In [215]:
scaler = MinMaxScaler()
scaler.fit(df_split)
scaled =  scaler.transform(df_split)

In [216]:
df_scaled = pd.DataFrame(scaled)
df_scaled

,0,1,2,3,4,5,6,7,8,9,...,198,199,200,201,202,203,204,205,206,207
0,0.541864,0.724921,0.541864,0.007545,0.479686,0.076311,0.078573,0.076030,0.067524,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.542008,0.725179,0.542008,0.008108,0.367214,0.064685,0.065357,0.064684,0.067524,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.539428,0.720542,0.539428,0.002015,0.487389,0.065274,0.066585,0.065273,0.067524,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.548084,0.727380,0.548084,0.012170,0.399573,0.084705,0.085974,0.084422,0.077170,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.547589,0.726338,0.547589,0.009467,0.495922,0.073667,0.073986,0.073665,0.077170,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8304,0.031597,0.930437,0.031597,0.253076,0.457017,0.028775,0.028376,0.028762,0.032154,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8305,0.602019,0.472964,0.602019,0.021169,0.562471,0.114261,0.115791,0.113723,0.090032,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8306,0.264453,0.532415,0.264453,0.028037,0.754238,0.164151,0.158473,0.164110,0.157556,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8307,0.681131,0.455405,0.681131,0.023001,0.490570,0.178045,0.174646,0.177222,0.151125,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [217]:
describe = df_scaled.describe().T
greater_005 = describe['std'] > 0.05
describe = describe[greater_005]
describe

,count,mean,std,min,25%,50%,75%,max
0,8309.0,0.512111,0.191102,0.0,0.331566,0.576297,0.657374,1.0
1,8309.0,0.693940,0.128894,0.0,0.664164,0.727587,0.777337,1.0
2,8309.0,0.512111,0.191102,0.0,0.331566,0.576297,0.657374,1.0
3,8309.0,0.041847,0.050069,0.0,0.010420,0.026325,0.056386,1.0
4,8309.0,0.594809,0.193594,0.0,0.462191,0.593192,0.746893,1.0
...,...,...,...,...,...,...,...,...
197,8309.0,0.009087,0.071996,0.0,0.000000,0.000000,0.000000,1.0
198,8309.0,0.015555,0.069865,0.0,0.000000,0.000000,0.000000,1.0
199,8309.0,0.009267,0.070920,0.0,0.000000,0.000000,0.000000,1.0
200,8309.0,0.004874,0.051521,0.0,0.000000,0.000000,0.000000,1.0


In [218]:
describe.to_csv('./Acute Oral Toxicity/temp2/Describe.csv')

In [219]:
filter = describe['std'] > 0.05
df_filtered = df_split.iloc[:,filter.index]
df_label = df.iloc[:, 0:1027]
# df_label
df_free = pd.concat([df_label, df_filtered], axis=1)
df_free

,ID,Canonical_SMILES,LD50_minLog,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_oxazole,fr_para_hydroxylation,fr_phos_ester,fr_piperdine,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_thiazole
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,-0.337242,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,-0.124939,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,-0.632201,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,0.257439,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00006,COc1ccc([N+](=O)[O-])cc1,0.361728,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8314,ref1_11885,c1ccccc1,0.521530,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8315,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,-1.522879,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8316,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,-1.036212,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8317,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,-0.008774,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [220]:
df_free.to_csv('./Acute Oral Toxicity/final/AOT_LD50_Model_Free.csv')

In [225]:
df = pd.read_csv('./Acute Oral Toxicity/temp2/AOT_EPA.csv')
df = process(df)
df_a=df[['ID','Canonical_SMILES','EPA_category']]
df_b=df.loc[:,0:207]
recolumn = []
for i in range(208):
    recolumn.append(f'{nms[i]}')
df_b.columns = recolumn
df = pd.concat([df_a,df_b],axis=1)
df

139.76036024093628


,ID,Canonical_SMILES,EPA_category,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,2.0,10.074698,-0.461975,10.074698,0.059630,0.463602,157.556,153.524,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,3.0,10.076965,-0.458889,10.076965,0.064074,0.359544,138.126,132.078,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,2.0,10.036149,-0.514444,10.036149,0.015926,0.470728,139.110,134.070,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,3.0,10.173125,-0.432528,10.173125,0.096173,0.389482,171.583,165.535,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,CNc1ccc([N+](=O)[O-])cc1,3.0,10.185702,-0.417222,10.185702,0.114815,0.517281,152.153,144.089,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11750,ref1_11885,c1ccccc1,3.0,2.000000,2.000000,2.000000,2.000000,0.442628,78.114,72.066,...,0,0,0,0,0,0,0,0,0,0
11751,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,1.0,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,...,0,0,0,0,0,0,0,0,0,0
11752,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,2.0,5.684815,-2.768105,5.684815,0.221566,0.717616,304.352,283.184,...,0,0,0,0,0,0,0,0,0,0
11753,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,3.0,12.278519,-3.690648,12.278519,0.181768,0.473671,327.572,309.428,...,0,0,0,0,0,0,0,0,0,0


In [226]:
df.to_csv('./Acute Oral Toxicity/temp2/AOT_EPA_DESCRS.csv')

In [227]:
mfps = pd.read_csv('./Acute Oral Toxicity/final/AOT_EPA_MFPS.csv')
descrs = pd.read_csv('./Acute Oral Toxicity/temp2/AOT_EPA_DESCRS.csv')
df_final = pd.merge(mfps,descrs)
df_final = df_final.iloc[:,1:]
df_final = df_final.dropna()
df_final

,ID,Canonical_SMILES,EPA_category,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,CNc1ccc([N+](=O)[O-])cc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11750,ref1_11885,c1ccccc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11751,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11752,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,2.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11753,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,3.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [228]:
df_final.to_csv('./Acute Oral Toxicity/final/AOT_EPA_Features.csv')

In [229]:
df = pd.read_csv('./Acute Oral Toxicity/final/AOT_EPA_Features.csv', index_col=0)
df_split = df.iloc[:,1027:]
df_split

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,10.074698,-0.461975,10.074698,0.059630,0.463602,157.556,153.524,156.993056,52,0,...,0,0,0,0,0,0,0,0,0,0
1,10.076965,-0.458889,10.076965,0.064074,0.359544,138.126,132.078,138.042927,52,0,...,0,0,0,0,0,0,0,0,0,0
2,10.036149,-0.514444,10.036149,0.015926,0.470728,139.110,134.070,139.026943,52,0,...,0,0,0,0,0,0,0,0,0,0
3,10.173125,-0.432528,10.173125,0.096173,0.389482,171.583,165.535,171.008706,58,0,...,0,0,0,0,0,0,0,0,0,0
4,10.185702,-0.417222,10.185702,0.114815,0.517281,152.153,144.089,152.058577,58,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11750,2.000000,2.000000,2.000000,2.000000,0.442628,78.114,72.066,78.046950,30,0,...,0,0,0,0,0,0,0,0,0,0
11751,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,219.945901,66,0,...,0,0,0,0,0,0,0,0,0,0
11752,5.684815,-2.768105,5.684815,0.221566,0.717616,304.352,283.184,304.101050,108,0,...,0,0,0,0,0,0,0,0,0,0
11753,12.278519,-3.690648,12.278519,0.181768,0.473671,327.572,309.428,326.000829,104,0,...,0,0,0,0,0,0,0,0,0,0


In [230]:
scaler = MinMaxScaler()
scaler.fit(df_split)
scaled =  scaler.transform(df_split)

In [237]:
df_scaled = pd.DataFrame(scaled)
describe = df_scaled.describe().T
greater_003 = describe['std'] > 0.03
describe = describe[greater_003]
describe

,count,mean,std,min,25%,50%,75%,max
0,11744.0,0.563802,0.173629,0.0,0.476743,0.619539,0.692533,1.0
1,11744.0,0.671999,0.115489,0.0,0.645479,0.702453,0.744822,1.0
2,11744.0,0.563802,0.173629,0.0,0.476743,0.619539,0.692533,1.0
3,11744.0,0.040326,0.052154,0.0,0.009180,0.024019,0.052947,1.0
4,11744.0,0.588569,0.202808,0.0,0.454557,0.594280,0.746662,1.0
...,...,...,...,...,...,...,...,...
201,11744.0,0.003023,0.040899,0.0,0.000000,0.000000,0.000000,1.0
202,11744.0,0.004215,0.046178,0.0,0.000000,0.000000,0.000000,1.0
205,11744.0,0.004726,0.051782,0.0,0.000000,0.000000,0.000000,1.0
206,11744.0,0.008342,0.033721,0.0,0.000000,0.000000,0.000000,1.0


In [238]:
describe.to_csv('./Acute Oral Toxicity/temp2/Describe_EPA.csv')

In [239]:
filter = describe['std'] > 0.03
df_filtered = df_split.iloc[:,filter.index]
df_label = df.iloc[:, 0:1027]
# df_label
df_free = pd.concat([df_label, df_filtered], axis=1)
df_free

,ID,Canonical_SMILES,EPA_category,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,CNc1ccc([N+](=O)[O-])cc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11750,ref1_11885,c1ccccc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11751,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11752,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,2.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11753,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,3.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [240]:
df_free.to_csv('./Acute Oral Toxicity/final/AOT_EPA_Model_Free.csv')

In [243]:
df = pd.read_csv('./Acute Oral Toxicity/temp2/AOT_GHS.csv')
df = process(df)
df_a=df[['ID','Canonical_SMILES','GHS_category']]
df_b=df.loc[:,0:207]
recolumn = []
for i in range(208):
    recolumn.append(f'{nms[i]}')
df_b.columns = recolumn
df = pd.concat([df_a,df_b],axis=1)
df

135.71563577651978


,ID,Canonical_SMILES,GHS_category,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,4.0,10.074698,-0.461975,10.074698,0.059630,0.463602,157.556,153.524,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,4.0,10.076965,-0.458889,10.076965,0.064074,0.359544,138.126,132.078,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,3.0,10.036149,-0.514444,10.036149,0.015926,0.470728,139.110,134.070,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,4.0,10.173125,-0.432528,10.173125,0.096173,0.389482,171.583,165.535,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,CNc1ccc([N+](=O)[O-])cc1,5.0,10.185702,-0.417222,10.185702,0.114815,0.517281,152.153,144.089,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11840,ref1_11885,c1ccccc1,5.0,2.000000,2.000000,2.000000,2.000000,0.442628,78.114,72.066,...,0,0,0,0,0,0,0,0,0,0
11841,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,2.0,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,...,0,0,0,0,0,0,0,0,0,0
11842,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,3.0,5.684815,-2.768105,5.684815,0.221566,0.717616,304.352,283.184,...,0,0,0,0,0,0,0,0,0,0
11843,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,4.0,12.278519,-3.690648,12.278519,0.181768,0.473671,327.572,309.428,...,0,0,0,0,0,0,0,0,0,0


In [244]:
df.to_csv('./Acute Oral Toxicity/temp2/AOT_GHS_DESCRS.csv')

In [260]:
mfps = pd.read_csv('./Acute Oral Toxicity/final/AOT_GHS_MFPS.csv')
descrs = pd.read_csv('./Acute Oral Toxicity/temp2/AOT_GHS_DESCRS.csv')
df_final = pd.merge(mfps,descrs)
df_final = df_final.iloc[:,1:]
df_final = df_final.dropna()
df_final

,ID,Canonical_SMILES,GHS_category,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,CNc1ccc([N+](=O)[O-])cc1,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11840,ref1_11885,c1ccccc1,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11841,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11842,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,3.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11843,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,4.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [261]:
df_final.to_csv('./Acute Oral Toxicity/final/AOT_GHS_Features.csv')

In [262]:
df = pd.read_csv('./Acute Oral Toxicity/final/AOT_GHS_Features.csv', index_col=0)
df_split = df.iloc[:,1027:]
df_split

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,10.074698,-0.461975,10.074698,0.059630,0.463602,157.556,153.524,156.993056,52,0,...,0,0,0,0,0,0,0,0,0,0
1,10.076965,-0.458889,10.076965,0.064074,0.359544,138.126,132.078,138.042927,52,0,...,0,0,0,0,0,0,0,0,0,0
2,10.036149,-0.514444,10.036149,0.015926,0.470728,139.110,134.070,139.026943,52,0,...,0,0,0,0,0,0,0,0,0,0
3,10.173125,-0.432528,10.173125,0.096173,0.389482,171.583,165.535,171.008706,58,0,...,0,0,0,0,0,0,0,0,0,0
4,10.185702,-0.417222,10.185702,0.114815,0.517281,152.153,144.089,152.058577,58,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11840,2.000000,2.000000,2.000000,2.000000,0.442628,78.114,72.066,78.046950,30,0,...,0,0,0,0,0,0,0,0,0,0
11841,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,219.945901,66,0,...,0,0,0,0,0,0,0,0,0,0
11842,5.684815,-2.768105,5.684815,0.221566,0.717616,304.352,283.184,304.101050,108,0,...,0,0,0,0,0,0,0,0,0,0
11843,12.278519,-3.690648,12.278519,0.181768,0.473671,327.572,309.428,326.000829,104,0,...,0,0,0,0,0,0,0,0,0,0


In [263]:
scaler = MinMaxScaler()
scaler.fit(df_split)
scaled =  scaler.transform(df_split)

In [264]:
df_scaled = pd.DataFrame(scaled)
describe = df_scaled.describe().T
greater_005 = describe['std'] > 0.03
describe = describe[greater_005]
describe

,count,mean,std,min,25%,50%,75%,max
0,11834.0,0.563792,0.173696,0.0,0.476767,0.619539,0.692530,1.0
1,11834.0,0.671896,0.115672,0.0,0.645210,0.702541,0.744814,1.0
2,11834.0,0.563792,0.173696,0.0,0.476767,0.619539,0.692530,1.0
3,11834.0,0.040326,0.052079,0.0,0.009195,0.024059,0.052937,1.0
4,11834.0,0.588211,0.202803,0.0,0.454324,0.593772,0.746477,1.0
...,...,...,...,...,...,...,...,...
201,11834.0,0.003000,0.040744,0.0,0.000000,0.000000,0.000000,1.0
202,11834.0,0.004183,0.046004,0.0,0.000000,0.000000,0.000000,1.0
205,11834.0,0.004732,0.051787,0.0,0.000000,0.000000,0.000000,1.0
206,11834.0,0.008492,0.034380,0.0,0.000000,0.000000,0.000000,1.0


In [265]:
describe.to_csv('./Acute Oral Toxicity/temp2/Describe_GHS.csv')

In [269]:
filter = describe['std'] > 0.03
df_filtered = df_split.iloc[:,filter.index]
df_label = df.iloc[:, 0:1027]
df_label
df_free = pd.concat([df_label, df_filtered], axis=1)
df_free

,ID,Canonical_SMILES,GHS_category,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,CNc1ccc([N+](=O)[O-])cc1,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11840,ref1_11885,c1ccccc1,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11841,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11842,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,3.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11843,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,4.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [270]:
df_free.to_csv('./Acute Oral Toxicity/final/AOT_GHS_Model_Free.csv')

# AMES Mutagenicity

In [271]:
df = pd.read_csv('./AMES Mutagenesis/final/AMES_Final.csv')
df = process(df)
df_a=df[['ID','Canonical_SMILES','Mutagenicity']]
df_b=df.loc[:,0:207]
recolumn = []
for i in range(208):
    recolumn.append(f'{nms[i]}')
df_b.columns = recolumn
df = pd.concat([df_a,df_b],axis=1)
df

91.63876342773438


,ID,Canonical_SMILES,Mutagenicity,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...,0,14.431073,-0.378575,14.431073,0.128988,0.184582,646.614,628.470,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00008,CC(=O)OC1(C(C)=O)CCC2C3C=C(Cl)C4=CC(=O)OCC4(C)...,0,12.729807,-1.064645,12.729807,0.073677,0.649284,406.906,379.690,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00009,Nc1nc(N)nc(N)n1,0,5.137222,0.041667,5.137222,0.041667,0.394014,126.123,120.075,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00010,Cc1ccc(N=Nc2c(O)ccc3ccccc23)c([N+](=O)[O-])c1,1,11.119127,-0.495723,11.119127,0.025727,0.410633,307.309,294.205,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00011,CC(C)CC(=O)Nc1snc2ccccc12,0,11.632299,0.059632,11.632299,0.059632,0.885112,234.324,220.212,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8629,ref4_08306,O=C(O)CCC(=O)OCC(N=C(O)C(Cl)Cl)C(O)c1ccc([N+](...,0,11.545085,-1.423786,11.545085,0.198546,0.128911,423.205,407.077,...,0,0,0,0,0,0,0,0,0,0
8630,ref4_08309,O=[N+]([O-])c1ccc(CNc2nc[nH]c3ncnc2-3)cc1,0,10.569682,-0.421734,10.569682,0.074516,0.552443,270.252,260.172,...,0,0,0,0,0,0,0,0,0,0
8631,ref4_08328,O=[N+]([O-])c1ccc(-[n+]2nc(-c3ccccc3)nn2-c2ccc...,0,10.923042,-0.422913,10.923042,0.030521,0.197257,470.250,457.146,...,0,0,0,0,1,0,0,0,0,0
8632,ref4_08330,O=C(C=Cc1cccc([N+](=O)[O-])c1)c1ccccc1,0,11.816116,-0.461947,11.816116,0.008620,0.362433,253.257,242.169,...,0,0,0,0,0,0,0,0,0,0


In [272]:
df.to_csv('./AMES Mutagenesis/temp/AMES_DESCRS.csv')

In [330]:
mfps = pd.read_csv('./AMES Mutagenesis/final/AMES_MFPS.csv')
descrs = pd.read_csv('./AMES Mutagenesis/temp/AMES_DESCRS.csv')
df_final = pd.merge(mfps,descrs)
df_final = df_final.iloc[:,1:]
df_final = df_final.dropna()
df_final

,ID,Canonical_SMILES,Mutagenicity,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00008,CC(=O)OC1(C(C)=O)CCC2C3C=C(Cl)C4=CC(=O)OCC4(C)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00009,Nc1nc(N)nc(N)n1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00010,Cc1ccc(N=Nc2c(O)ccc3ccccc23)c([N+](=O)[O-])c1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00011,CC(C)CC(=O)Nc1snc2ccccc12,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8629,ref4_08306,O=C(O)CCC(=O)OCC(N=C(O)C(Cl)Cl)C(O)c1ccc([N+](...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8630,ref4_08309,O=[N+]([O-])c1ccc(CNc2nc[nH]c3ncnc2-3)cc1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8631,ref4_08328,O=[N+]([O-])c1ccc(-[n+]2nc(-c3ccccc3)nn2-c2ccc...,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8632,ref4_08330,O=C(C=Cc1cccc([N+](=O)[O-])c1)c1ccccc1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [331]:
df_final.to_csv('./AMES Mutagenesis/final/AMES_Features.csv')

In [332]:
df = pd.read_csv('./AMES Mutagenesis/final/AMES_Features.csv', index_col=0)
df_split = df.iloc[:,1027:]
df_split

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,14.431073,-0.378575,14.431073,0.128988,0.184582,646.614,628.470,646.116486,232,0,...,0,0,0,0,0,0,0,0,0,0
1,12.729807,-1.064645,12.729807,0.073677,0.649284,406.906,379.690,406.154702,152,0,...,0,0,0,0,0,0,0,0,0,0
2,5.137222,0.041667,5.137222,0.041667,0.394014,126.123,120.075,126.065394,48,0,...,0,0,0,0,0,0,0,0,0,0
3,11.119127,-0.495723,11.119127,0.025727,0.410633,307.309,294.205,307.095691,114,0,...,0,0,0,0,0,0,0,0,0,0
4,11.632299,0.059632,11.632299,0.059632,0.885112,234.324,220.212,234.082684,84,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8629,11.545085,-1.423786,11.545085,0.198546,0.128911,423.205,407.077,422.028371,148,0,...,0,0,0,0,0,0,0,0,0,0
8630,10.569682,-0.421734,10.569682,0.074516,0.552443,270.252,260.172,270.086524,100,0,...,0,0,0,0,0,0,0,0,0,0
8631,10.923042,-0.422913,10.923042,0.030521,0.197257,470.250,457.146,470.010849,132,0,...,0,0,0,0,1,0,0,0,0,0
8632,11.816116,-0.461947,11.816116,0.008620,0.362433,253.257,242.169,253.073893,94,0,...,0,0,0,0,0,0,0,0,0,0


In [335]:
scaler = MinMaxScaler()
scaler.fit(df_split)
scaled =  scaler.transform(df_split)
df_scaled = pd.DataFrame(scaled)
describe = df_scaled.describe().T
greater_005 = describe['std'] > 0.035
describe = describe[greater_005]
describe

,count,mean,std,min,25%,50%,75%,max
0,8620.0,0.504425,0.190906,0.0,0.380847,0.565193,0.636324,1.0
1,8620.0,0.629146,0.092896,0.0,0.604497,0.639282,0.688869,1.0
2,8620.0,0.504425,0.190906,0.0,0.380847,0.565193,0.636324,1.0
3,8620.0,0.045134,0.055159,0.0,0.009364,0.025819,0.064820,1.0
4,8620.0,0.533252,0.184700,0.0,0.411108,0.534905,0.657139,1.0
...,...,...,...,...,...,...,...,...
201,8620.0,0.003712,0.060819,0.0,0.000000,0.000000,0.000000,1.0
203,8620.0,0.007193,0.066446,0.0,0.000000,0.000000,0.000000,1.0
205,8620.0,0.005104,0.051405,0.0,0.000000,0.000000,0.000000,1.0
206,8620.0,0.009757,0.053140,0.0,0.000000,0.000000,0.000000,1.0


In [336]:
describe.to_csv('./AMES Mutagenesis/temp/Describe_AMES.csv')

In [337]:
filter = describe['std'] > 0.035
df_filtered = df_split.iloc[:,filter.index]
df_label = df.iloc[:, 0:1027]
# df_label
df_free = pd.concat([df_label, df_filtered], axis=1)
df_free

,ID,Canonical_SMILES,Mutagenicity,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_phos_ester,fr_piperdine,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_term_acetylene,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00008,CC(=O)OC1(C(C)=O)CCC2C3C=C(Cl)C4=CC(=O)OCC4(C)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00009,Nc1nc(N)nc(N)n1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00010,Cc1ccc(N=Nc2c(O)ccc3ccccc23)c([N+](=O)[O-])c1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00011,CC(C)CC(=O)Nc1snc2ccccc12,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8629,ref4_08306,O=C(O)CCC(=O)OCC(N=C(O)C(Cl)Cl)C(O)c1ccc([N+](...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8630,ref4_08309,O=[N+]([O-])c1ccc(CNc2nc[nH]c3ncnc2-3)cc1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8631,ref4_08328,O=[N+]([O-])c1ccc(-[n+]2nc(-c3ccccc3)nn2-c2ccc...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8632,ref4_08330,O=C(C=Cc1cccc([N+](=O)[O-])c1)c1ccccc1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [338]:
df_free.to_csv('./AMES Mutagenesis/final/AMES_Model_Free.csv')

# hERG

In [284]:
df = pd.read_csv('./hERG/final/hERG_Final.csv')
df = process(df)
df_a=df[['ID','Canonical_SMILES','hERG']]
df_b=df.loc[:,0:207]
recolumn = []
for i in range(208):
    recolumn.append(f'{nms[i]}')
df_b.columns = recolumn
df = pd.concat([df_a,df_b],axis=1)
df

266.07732796669006


,ID,Canonical_SMILES,hERG,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,Brc1ccc(-n2ccc3ccncc32)cc1,1,4.158718,1.088738,4.158718,1.088738,0.659239,273.133,264.061,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Brc1cnc(NCC2CC2)nc1,1,4.108912,0.726944,4.108912,0.726944,0.859843,228.093,218.013,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,C(#Cc1cc(-c2[nH]nc3c2Cc2cc(Cn4ccnc4)ccc2-3)cs1...,1,5.655430,0.368471,5.655430,0.368471,0.360150,448.551,428.391,...,0,0,0,0,0,0,0,1,0,0
3,ref1_00004,C(#Cc1cc(-c2[nH]nc3c2Cc2cc(Cn4cncn4)ccc2-3)cs1...,1,5.651524,0.364026,5.651524,0.364026,0.384762,449.539,430.387,...,0,0,0,0,0,0,0,1,0,0
4,ref1_00005,C1=C/COCc2cc(ccc2OCCN2CCCC2)Nc2nccc(n2)-c2ccc(...,1,6.157068,0.455430,6.157068,0.455430,0.527825,478.618,448.378,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13897,ref4_10415,CC1(C)CN(c2ccc3c(c2)[C@]2(COC(N)=N2)c2cc(-c4cn...,0,6.333457,-0.781288,6.333457,0.180700,0.648107,443.507,418.307,...,0,0,0,0,0,0,0,0,0,0
13898,ref4_10416,NC1=N[C@@]2(CO1)c1cc(-c3cncnc3)ccc1Oc1ccc(N3CC...,0,6.292514,-0.762952,6.292514,0.184281,0.687505,415.453,394.285,...,0,0,0,0,0,0,0,0,0,0
13899,ref4_10417,CC(C)(O)COc1ccc2c(c1)[C@]1(COC(N)=N1)c1cc(-c3c...,0,10.007808,-0.955956,10.007808,0.120426,0.669746,418.453,396.277,...,0,0,0,0,0,0,0,0,0,0
13900,ref4_10418,CC1(COc2ccc3c(c2)[C@]2(COC(N)=N2)c2cc(-c4cncnc...,0,6.250589,-0.817956,6.250589,0.044044,0.677937,430.464,408.288,...,0,0,0,0,0,0,0,0,0,0


In [285]:
df.to_csv('./hERG/temp/hERG_DESCRS.csv')

In [286]:
mfps = pd.read_csv('./hERG/final/hERG_MFPS.csv')
descrs = pd.read_csv('./hERG/temp/hERG_DESCRS.csv')
df_final = pd.merge(mfps,descrs)
df_final = df_final.iloc[:,1:]
df_final = df_final.dropna()
df_final

,ID,Canonical_SMILES,hERG,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,Brc1ccc(-n2ccc3ccncc32)cc1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Brc1cnc(NCC2CC2)nc1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,C(#Cc1cc(-c2[nH]nc3c2Cc2cc(Cn4ccnc4)ccc2-3)cs1...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,ref1_00004,C(#Cc1cc(-c2[nH]nc3c2Cc2cc(Cn4cncn4)ccc2-3)cs1...,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,ref1_00005,C1=C/COCc2cc(ccc2OCCN2CCCC2)Nc2nccc(n2)-c2ccc(...,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13897,ref4_10415,CC1(C)CN(c2ccc3c(c2)[C@]2(COC(N)=N2)c2cc(-c4cn...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13898,ref4_10416,NC1=N[C@@]2(CO1)c1cc(-c3cncnc3)ccc1Oc1ccc(N3CC...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13899,ref4_10417,CC(C)(O)COc1ccc2c(c1)[C@]1(COC(N)=N1)c1cc(-c3c...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13900,ref4_10418,CC1(COc2ccc3c(c2)[C@]2(COC(N)=N2)c2cc(-c4cncnc...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [287]:
df_final.to_csv('./AMES Mutagenesis/final/AMES_Features.csv')

In [322]:
df = pd.read_csv('./AMES Mutagenesis/final/AMES_Features.csv', index_col=0)
df_split = df.iloc[:,1027:]
df_split

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,4.158718,1.088738,4.158718,1.088738,0.659239,273.133,264.061,271.994910,78,0,...,0,0,0,0,0,0,0,0,0,0
1,4.108912,0.726944,4.108912,0.726944,0.859843,228.093,218.013,227.005809,64,0,...,0,0,0,0,0,0,0,0,0,0
2,5.655430,0.368471,5.655430,0.368471,0.360150,448.551,428.391,448.135782,160,0,...,0,0,0,0,0,0,0,1,0,0
3,5.651524,0.364026,5.651524,0.364026,0.384762,449.539,430.387,449.131031,160,0,...,0,0,0,0,0,0,0,1,0,0
4,6.157068,0.455430,6.157068,0.455430,0.527825,478.618,448.378,478.203862,178,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13897,6.333457,-0.781288,6.333457,0.180700,0.648107,443.507,418.307,443.195740,168,0,...,0,0,0,0,0,0,0,0,0,0
13898,6.292514,-0.762952,6.292514,0.184281,0.687505,415.453,394.285,415.164440,156,0,...,0,0,0,0,0,0,0,0,0,0
13899,10.007808,-0.955956,10.007808,0.120426,0.669746,418.453,396.277,418.164105,158,0,...,0,0,0,0,0,0,0,0,0,0
13900,6.250589,-0.817956,6.250589,0.044044,0.677937,430.464,408.288,430.164105,162,0,...,0,0,0,0,0,0,0,0,0,0


In [326]:
scaler = MinMaxScaler()
scaler.fit(df_split)
scaled =  scaler.transform(df_split)
df_scaled = pd.DataFrame(scaled)
describe = df_scaled.describe().T
greater_005 = describe['std'] > 0.07
describe = describe[greater_005]
describe

,count,mean,std,min,25%,50%,75%,max
0,13901.0,0.685355,0.188535,0.0,0.677920,0.745069,0.788080,1.0
1,13901.0,0.711417,0.154018,0.0,0.554180,0.780604,0.817072,1.0
2,13901.0,0.685355,0.188535,0.0,0.677920,0.745069,0.788080,1.0
3,13901.0,0.080493,0.093364,0.0,0.017635,0.048840,0.104825,1.0
4,13901.0,0.579020,0.207422,0.0,0.424444,0.576396,0.742078,1.0
...,...,...,...,...,...,...,...,...
200,13901.0,0.016689,0.092382,0.0,0.000000,0.000000,0.000000,1.0
202,13901.0,0.013165,0.113983,0.0,0.000000,0.000000,0.000000,1.0
203,13901.0,0.020538,0.101033,0.0,0.000000,0.000000,0.000000,1.0
205,13901.0,0.018920,0.096531,0.0,0.000000,0.000000,0.000000,1.0


In [327]:
describe.to_csv('./hERG/temp/Describe_hERG.csv')

In [328]:
filter = describe['std'] > 0.07
df_filtered = df_split.iloc[:,filter.index]
df_label = df.iloc[:, 0:1027]
# df_label
df_free = pd.concat([df_label, df_filtered], axis=1)
df_free

,ID,Canonical_SMILES,hERG,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_piperzine,fr_priamide,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_tetrazole,fr_thiazole,fr_thiophene,fr_urea
0,ref1_00001,Brc1ccc(-n2ccc3ccncc32)cc1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,ref1_00002,Brc1cnc(NCC2CC2)nc1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,C(#Cc1cc(-c2[nH]nc3c2Cc2cc(Cn4ccnc4)ccc2-3)cs1...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,ref1_00004,C(#Cc1cc(-c2[nH]nc3c2Cc2cc(Cn4cncn4)ccc2-3)cs1...,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,ref1_00005,C1=C/COCc2cc(ccc2OCCN2CCCC2)Nc2nccc(n2)-c2ccc(...,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13897,ref4_10415,CC1(C)CN(c2ccc3c(c2)[C@]2(COC(N)=N2)c2cc(-c4cn...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13898,ref4_10416,NC1=N[C@@]2(CO1)c1cc(-c3cncnc3)ccc1Oc1ccc(N3CC...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13899,ref4_10417,CC(C)(O)COc1ccc2c(c1)[C@]1(COC(N)=N1)c1cc(-c3c...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13900,ref4_10418,CC1(COc2ccc3c(c2)[C@]2(COC(N)=N2)c2cc(-c4cncnc...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [329]:
df_free.to_csv('./hERG/final/hERG_Model_Free.csv')

# Confirm

In [349]:
dili = pd.read_csv('./Model-Free-Final/DILI_Model_Free.csv',index_col=0)
dili.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2226 entries, 0 to 2260
Columns: 1208 entries, ID to fr_urea
dtypes: float64(95), int64(1111), object(2)
memory usage: 20.5+ MB


In [350]:
AOT_LD = pd.read_csv('./Model-Free-Final/AOT_LD50_Model_Free.csv',index_col=0)
AOT_LD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8309 entries, 0 to 8318
Columns: 1170 entries, ID to fr_thiazole
dtypes: float64(84), int64(1084), object(2)
memory usage: 74.2+ MB


In [351]:
AOT_EPA = pd.read_csv('./Model-Free-Final/AOT_EPA_Model_Free.csv', index_col=0)
AOT_EPA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11744 entries, 0 to 11754
Columns: 1172 entries, ID to fr_urea
dtypes: float64(72), int64(1098), object(2)
memory usage: 105.1+ MB


In [353]:
AOT_EPA['EPA_category'].value_counts()

3.0    5861
2.0    2632
4.0    2271
1.0     980
Name: EPA_category, dtype: int64

In [354]:
AOT_GHS = pd.read_csv('./Model-Free-Final/AOT_GHS_Model_Free.csv', index_col=0)
AOT_GHS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11834 entries, 0 to 11844
Columns: 1172 entries, ID to fr_urea
dtypes: float64(72), int64(1098), object(2)
memory usage: 105.9+ MB


In [355]:
AOT_GHS['GHS_category'].value_counts()

5.0    5082
4.0    4175
3.0    1597
2.0     752
1.0     228
Name: GHS_category, dtype: int64

In [356]:
AMES = pd.read_csv('./Model-Free-Final/AMES_Model_Free.csv', index_col=0)
AMES.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8620 entries, 0 to 8633
Columns: 1157 entries, ID to fr_unbrch_alkane
dtypes: float64(65), int64(1090), object(2)
memory usage: 76.2+ MB


In [357]:
AMES['Mutagenicity'].value_counts()

1    4828
0    3792
Name: Mutagenicity, dtype: int64

In [358]:
hERG = pd.read_csv('./Model-Free-Final/hERG_Model_Free.csv', index_col=0)
hERG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13901 entries, 0 to 13901
Columns: 1199 entries, ID to fr_urea
dtypes: float64(99), int64(1098), object(2)
memory usage: 127.3+ MB


In [359]:
hERG['hERG'].value_counts()

1    8112
0    5789
Name: hERG, dtype: int64